### Creación de tablas.

#### `CREATE TABLE`

Para crear una tabla, de momento, necesitamos declarar bajo el _key word_ `CREATE TABLE` y/o el nombre del esquema, más el nombre de la tabla (`esquema.tabla`). 

El _statement_ anterior ordenará solamente la construcción de la tabla. `INSERT INTO` nos ayudará a la incersión de los atributos dentro de la tabla, pero antes es necesario declara el nombre y el tipo de dato para cada uno de ellos con ayuda de `VALUES`. 

* `id` **serial PRIMARY KEY**
* `num` **integer**
* `data` **varchar**

Para mantener la conección abierta, cada vez que necesitemos crear un nuevo _query_, con la base de datos, siempre ejecutamos por separado la llamada de la librería **psycopg2** y la llamada de la base de datos. Así que cada vez que que realicemos `cur.execute` y `conn.close` la conección seguirá activa por que la definimos desde un principio.

In [11]:
import pandas as pd
import psycopg2.extras
conn = psycopg2.connect("dbname='test' user='test' host='/tmp/'")
cur = conn.cursor()

In [ ]:
cur.execute(
    """
    CREATE TABLE postgis.tabla_psycopg (id serial PRIMARY KEY, num integer, data varchar);
    INSERT INTO postgis.tabla_psycopg VALUES (1, 2, 'mi_primera_tabla');
    """
)
conn.commit()
cur.close()
conn.close()

Lo anterior habrá creado la tabla `tabla_psycopg` con los atributos descritos dentro de la base de datos `tes` en el esquema `postgis`.

Para este punto necesitamos manejar ciertas habilidades un poco avanzadas para poder trabajar con las tablas de los ejercicios. De tal modo que se menciona, paso a paso, el procedimiento para realizar la carga de un archivo _.sql_ que contiene el script para crear las tablas dentro de una base de datos cuyo esquema nombraremos como `intro_sql`.

Es necesario recrear los siguientes comandos:  

* En una terminal ejecutamos el comando `psql test test` el cual nos conectara con la base de datos `test` y bajo el usuario `test`. Esta es una forma directa de conectarnos a cualquier base de datos desde terminal sin tener que pasar por el usuario de `postgres=#`. En el ejemplo anterior lo realizamos con **psycopg2** por tal motivo no ejecutamos este comando.  
* Una vez conectados a la base de datos `test` como usuaruo `test` tiramos la línea `CREATE SCHEMA intro_sql`

Ahora ingresamos a la páginoa [web](https://assets.datacamp.com/production/repositories/641/datasets/d39d51ab6615b8a7f36681ee88490978fb095193/films.sql) y tenemos dos opciones para descargar el archivo: via `wget` (_terminal_) o dando click derecho y descargando el archivo _.sql_.

Con el archivo descargado ahora necesitamos editarlo. Podemos hacerlo desde terminal vía `gedit` (_terminal_) o con algún editor de textos.

Dentro del archivo será necesario ubicar la línea `SET search_path = public, pg_catalog;` y colocar en `search_path = intro_sql`.

Parte del archivo se muestra a continuación:

```sql
CREATE EXTENSION IF NOT EXISTS plpgsql WITH SCHEMA pg_catalog;


--
-- Name: EXTENSION plpgsql; Type: COMMENT; Schema: -; Owner:
--

COMMENT ON EXTENSION plpgsql IS 'PL/pgSQL procedural language';


SET search_path = intro_sql, pg_catalog;

SET default_tablespace = '';
```

Guardamos cambios y regresamos a una _terminal_ para poder ejecutar el comando `psql -U test test < /ruta/del/archivo/films.sql`.

Durante la ejecución del comando se arrojarán algunos errores como los siguientes, pero no hay por que hacer caso. La salida del comando será algo parecido a las siguientes líneas:

```sql
ERROR:  role "colinricardo" does not exist
CREATE TABLE
ERROR:  role "colinricardo" does not exist
CREATE TABLE
ERROR:  role "colinricardo" does not exist
CREATE TABLE
ERROR:  role "colinricardo" does not exist
COPY 4968
COPY 8397
COPY 4968
COPY 19791
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
```

Si ejecutamos en _Postgres_, de nuevo vía _terminal_, el comando `\dt. intro_sql`, dentro de la base de datos `test` veremos algo similar a esto:

```sql

psql test test -- entramos a la base de datos desde terminal.

test=# \dt intro_sql.

List of relations
  Schema   |  Name   | Type  | Owner 
-----------+---------+-------+-------
 intro_sql | films   | table | test
 intro_sql | people  | table | test
 intro_sql | reviews | table | test
 intro_sql | roles   | table | test
```
las tablas han sido cargadas en el esquema `intro_sql`.

### Selección de columnas.

#### `SELECT` 

Nos concentraremos en la consulta de la información a las tablas de nuestro nuevo esquema. Para ello es necesario construir la sintaxis que realizará esta tarea, también conocido como _"queriying"_. 

Un _"Query"_ es la construcción de la sintaxis (_statement_), en _**SQL**_, para poder obtener una respuesta a la consulta realizada en los datos de una tabla, o para generar nueva información, o para actualizar una tabla preexistente. En _**SQL**_ podemos seleccionar una tabla usando las _keys words_. **SELECT** y **FROM**, en mayúsculas como buena práctica, son algunas de ellas.

Por ejemplo. La siguiente sintaxis (_statement_) selecciona la columna `name` de la tabla `people`:

```sql
SELECT name FROM people;
```

_—Para poder realizar la elección de de múltiples columnas bastará con especificar el nombre de cada una de ellas. Por ejemplo:—_

```sql
SELECT name, birthdate FROM people;
```

_—O en su lugar seleccionar todas los atributos de la tabla utilizando `*`:—_

```sql
SELECT * FROM people;
```

_—Para limitar el número de elementos (_renglones_) de una consulta utilizamos la _key word_ `LIMIT`:—_

```sql
SELECT * FROM people LIMIT 10;
```

_—Si el resultado de nuestra consulta presume duplicados, `DISTINCT` selecciona solo los records **únicos** en un atributo:—_

```sql
SELECT DISTINCT rol FROM roles;
```

_—Si quicieramos realizar el conteo de valores existentes en un atributo, `COUNT` retorna el número de elementos:—_

```sql
SELECT COUNT(*) FROM people;
```

_—Mientras tanto, si necesitamos realizar un conteo de valores únicos _**non-missing values**_, a diferencia de utilizar `*`, `COUNT(DISTINCT atributo)` nos ayuda a realizar esta tarea;—_

```sql
SELECT COUNT(DISTINCT birthdate) FROM people;
```

Todas las consultas anteriores son ejemplos de _querys_ a las tablas dentro del esquema `public`. Si no especificamos donde guardar nuestra información, dentro de una base de datos, estas se almacenarán dentro del esquema `public` por default.

De tal manera que el _statement_ de las consultas para las tablas, dentro de un esquemas, difiere un poco. Por ejemplo: `SELECT tabla.columna FROM esquema.tabla;`

### Condiciones.

#### `WHERE`

En **SQL**, los siguientes operadores nos permiten hacer filtros a los atributos mediante el uso de texto o números. Para ello se antepone la keyword `WHERE` antes de su uso.

* `=` igual.
* `<>` no igual.
* `<` menos que.
* `>` mayor que.
* `<=` menor o igual.
* `>=` mayor que.

Por ejemplo: podemos filtrar, de la tabla `films`, en el atributo `title`, la película `Metropolis`. 

Para este ejemplo regresaremos, en un _Dataframe_, el _Query_ de la consulta realizada en _Posgres_. Para ello es necesario importar la librería de _pandas_ y utilizar el método de _psycopg2_ `extras` para convertir el **query** en un objeto de **python** en un _Dataframe_.

`fetchall` nos regresa todos los elementos del **query** en el objeto `data` y es este, quién dentro de un `for`, pasa los records de la consulta al _Dataframe_. 

In [6]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT * FROM intro_sql.films WHERE title = 'Metropolis';
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,id,title,release_year,country,duration,language,certification,gross,budget
0,4,Metropolis,1927,Germany,145,German,Not Rated,26435,6000000


_—Para consultar el número de películas que se estrenaron antes del año 2000 la consulta sería de la sigueinte manera:—_
```sql
SELECT COUNT(*) FROM intro_sql.films WHERE release_year<2000
```

_—La consulta para ver todos los detalles de las películas estrenadas en el 2016:—_
```sql
SELECT * FROM intro_sql.films WHERE release_year = 2016
```

_—Para conocer el título y el año de las películas estrenadas despues del año 2000:—_
```sql
SELECT title, release_year FROM intro_sql.films WHERE release_year>2000
```

Ahora podemos realizar la consulta ingresando texto como condición. Por ejemplo: podemos realizar la consulta a la tabla `films`, en el atributo `country`, para poder encontrar las películas realizadas en `China`:

In [7]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT films.title FROM intro_sql.films WHERE country = 'China';
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,title
0,The Last Emperor
1,Hero
2,Hero
3,House of Flying Daggers
4,The Promise
5,Crazy Stone
6,Curse of the Golden Flower
7,The Painted Veil
8,Red Cliff
9,Three Kingdoms: Resurrection of the Dragon


_—Todos los datos de las películas realisadas en lengua francesa (`French`), de la tabla `films`—:_
```sql
SELECT * FROM films WHERE language = 'French' 
```
_—Datos completos de la persona que nacio el día `1974-11-11`, de la tabla `people`:—_
```sql
SELECT name, birthdate FROM people WHERE birthdate = '1974-11-11' 
```
_—El número de películas realizadas en lengua indú (`Hindi`), de la tabla `films`:—_
```sql
SELECT COUNT(language) FROM films WHERE language = 'Hindi' 
```

#### `WHERE AND`

En otras muchas ocasiónes necesitamos declarar varias condiciones. Podemos combinar el uso de `WHERE` con `AND` para poder realizar consultas con varias condiciones. Por ejemplo:

—_Seleccionar el título y el año de estreno de las películas, en español, antes del 2000_:—
```sql
SELECT title, release_year FROM films WHERE  language='Spanish' AND release_year < 2000;
```

Podemos observar que cada vez que insertamos `AND` es necesario referirnos a la columna de la consulta. Un ejemplo de una mala sintaxis sería la siguiente:

```sql
SELECT title
FROM films
WHERE release_year > 1994 AND < 2000;
```
En el _statement_ anterior ~~~`WHERE release_year > 1994 AND < 2000`~~~, después de `AND` no se agrega de nueva cuenta el atributo `release_year`.

La siguirnte consulta tratara de arrojarnos los detalles de las películas en español, que fueron estrenadas después del 2000 y antes del 2010:

In [3]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT * FROM intro_sql.films WHERE language='Spanish' AND films.release_year > 2000 AND release_year < 2010;
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,id,title,release_year,country,duration,language,certification,gross,budget
0,1695,Y Tu Mamá También,2001,Mexico,106,Spanish,R,13622333.0,2000000.0
1,1757,El crimen del padre Amaro,2002,Mexico,118,Spanish,R,5709616.0,1800000.0
2,1807,Mondays in the Sun,2002,Spain,113,Spanish,R,146402.0,4000000.0
3,2173,Live-In Maid,2004,Argentina,83,Spanish,Unrated,NaN,800000.0
4,2175,Maria Full of Grace,2004,Colombia,101,Spanish,R,6517198.0,3000000.0
5,2246,The Holy Girl,2004,Argentina,106,Spanish,R,304124.0,1400000.0
6,2263,The Sea Inside,2004,Spain,125,Spanish,PG-13,2086345.0,10000000.0
7,2458,The Legend of Zorro,2005,USA,129,Spanish,PG,45356386.0,75000000.0
8,2542,Captain Alatriste: The Spanish Musketeer,2006,Spain,145,Spanish,None,NaN,24000000.0
9,2646,Pan's Labyrinth,2006,Spain,112,Spanish,R,37623143.0,13500000.0


### `WHERE` `AND` `OR`

Imaginemos que necesitamos tener varias combinaciones entre condiciones `WHERE`. Para ello el operador `OR` nos ayuda a la consulta junto con `WHERE` (necesariamente) y `AND`. 

Parte de diferencia radica en que:

* `AND` establece múltiples condiciones con la condición `WHERE` y las consultas `SELECT`, `UPDATE` y `DELETE`. Todos los resultados en la consulta tuvieron que satisfacer las condiciones propuestas. Con un elemento que no lo haga `AND` np podrá ejecutarse.

* Cuando usamos `OR` al menos uno de los elementos dentro del resultado de la consulta quedó satisfecho. Es decir, no necesariamente la proposición en todas las condiciones se debe cumplir para poder llevarse a cabo toda la consulta.

_—La consulta a la tabla `films` para encontrar los titulos de las películas que fueron estrenadas en `1994` o `1995`, y además donde la clasificación es `R` o `PG`, sería de la siguiente forma:—_
```sql
SELECT films.title FROM intro_sql.films WHERE (release_year = 1994 OR release_year = 1995)
AND (certification = 'PG' OR certification = 'R');
```
_—El siguiente caso nos presenta los título de las películas y la fecha de su lanzamiento, donde; el año de estreno son los 90's, el lenguaje es Español o Frances y además la recaudación total supero los 2M:—_

In [8]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT films.title, films.release_year FROM intro_sql.films 
    WHERE (release_year>=1990 AND release_year<2000)
    AND (language='Spanish' OR language='French')
    AND (gross>2000000);
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,title,release_year
0,El Mariachi,1992
1,The Red Violin,1998


### `BETWEEN`

Observemos el siguiente _query_:

```sql
SELECT title
FROM films
WHERE release_year
BETWEEN 1994 AND 2000;
```
Se utiliza `BETWEEN` para acotar las cantidades `1994` y `2000` en lugar de la froma ~~`AND (release_year>=1994 or release_year<2000)`~~. Otra cosa que hay que notar es que `BETWEEN` es inclusivo. Esto quiere decir que las cantidades `1994` y `2000` se incluirpan en el resultado.

_—Imaginemos que tenemos que consultar: qué películas en idioma Español o Frances, estrenadas entre 1990 y 2000, recaudaron máss de $100,000,000:—_

In [12]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT films.title, films.release_year
    FROM intro_sql.films
    WHERE release_year BETWEEN 1990 AND 2000
    AND budget> 100000000
    AND (language='Spanish' or language='French');
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,title,release_year
0,Les couloirs du temps: Les visiteurs II,1998
1,Tango,1998
